# EDA & Data Cleaning - [Your Project Name Here]

## Local Code Imports - Do not delete

In [3]:
# DO NOT REMOVE THESE
%load_ext autoreload
%autoreload 2

In [4]:
# DO NOT REMOVE This
%reload_ext autoreload

In [5]:
## Uncomment to filter warnings
#import warnings
#warnings.filterwarnings('ignore')

In [6]:
## DO NOT REMOVE
## import local src module -
## src in this project will contain all your local code
## clean_data.py, model.py, visualize.py, custom.py
from src import make_data as mk
from src import visualize as viz
from src import model as mdl
from src import pandas_operators as po

def test_src():
    mk.test_make_data()
    viz.test_viz()
    mdl.test_model()
    po.test_pandas()
    
    return 1

In [7]:
test_src()

In make_data
In Visualize
In Model
In pandas ops


1

## Code Imports

In [8]:
## Some standard imports
import numpy as np
import pandas as pd

# import pandas_profiling as pp  # uncomment if you want to use pandas_profiling
## must have pandas_profiling installed locally.
##
## pip install pandas-profiling
##

import matplotlib.pyplot as plt
import seaborn as sns

# Project Overview


The purpose of this project is to create a recommendation system that overcomes the echo chamber problem. The echo chamber problem occurs when additional information provided to a recommendation system results in a decrease in the diversity of recommendations for a given user. The result is a recommendation system that reinforces a user’s recorded preferences and fails to provide recommendations outside the user’s typical purview.

In order to overcome the echo chamber problem, this project builds a system that provides additional recommendations based on the preferences of others who are different, but not too different, from the user. These additional recommendations are identified by clustering users based on the latent user features from an ALS decomposition of a ratings matrix. Once clusters are created, the top-rated items from users in each cluster can be identified. The top-rated items in each cluster form the set of items from which the recommendations will be extracted. To get recommendations for a user, the following steps occur:

The distance between clusters is measured,
A user is classified into a cluster,
From the nearest cluster to the user’s cluster, a random selection of items is recommended to the user.
Step three can be repeated for the k nearest clusters. In such a case, the proportion of total recommendations selected from a cluster is weighted by the cluster’s distance from the user’s cluster. (For example, for k=3 and a total of 10 recommendations, 5 recommendations may be selected from the nearest cluster, 3 from the second nearest, and 2 from the third nearest.)

## Data Description

The project utilizes the MovieLens dataset (Harper & Konstan, 2015). The MovieLens dataset is an open source data set containing 27,753,444 movie ratings from 283,228 users for 58,098 movies. The ratings are on a five-star scale range from 0.5 stars to 5 stars in 0.5 star increments. The files include data from January 09, 1995 and September 26, 2018. The data set includes a random sample of users with at least 1 movie rating.

### Data Dictionary

userId - A unique identifier for each user in the dataset

movieId - A unique identifier for each movie in the dataset

rating - A user's rating for a given movie. The ratings utilize a five star system ranging from 0.5 stars to 5 stars in 0.5 star increments. 

timestamp - The timestamp is the time at which the rating was made represented as seconds since midnight January 1, 1970 (UTC).

# EDA

## Load Data - All sources

In [9]:
ratings_df = pd.read_csv('../data/raw/ratings.csv')

## Data Summary

In [10]:
ratings_df.describe()

,userId,movieId,rating,timestamp
count,2.775344e+07,2.775344e+07,2.775344e+07,2.775344e+07
mean,1.419420e+05,1.848800e+04,3.530445e+00,1.193122e+09
std,8.170740e+04,3.510263e+04,1.066353e+00,2.160482e+08
min,1.000000e+00,1.000000e+00,5.000000e-01,7.896520e+08
25%,7.117600e+04,1.097000e+03,3.000000e+00,9.986053e+08
50%,1.420220e+05,2.716000e+03,3.500000e+00,1.174256e+09
75%,2.124590e+05,7.150000e+03,4.000000e+00,1.422744e+09
max,2.832280e+05,1.938860e+05,5.000000e+00,1.537945e+09


In [11]:
ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27753444 entries, 0 to 27753443
Data columns (total 4 columns):
userId       int64
movieId      int64
rating       float64
timestamp    int64
dtypes: float64(1), int64(3)
memory usage: 847.0 MB


## Detailed Analysis & Results

In [12]:
ratings_df.isna().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

In [13]:
ratings_df.rating.value_counts()

4.0    7394710
3.0    5515668
5.0    4071135
3.5    3404360
4.5    2373550
2.0    1850627
2.5    1373419
1.0     886233
0.5     442388
1.5     441354
Name: rating, dtype: int64

In [14]:
ratings_df['userId'].nunique()

283228

In [15]:
(ratings_df.groupby(['userId'])['movieId'].agg('count')<10).sum()

39570

# Data Cleaning

The following cell removes the timestamp column since it will not be used in this recommendation system.

In [16]:
ratings_df.drop(['timestamp'], axis=1, inplace=True)

The next cell creates a series of boolean values indicating whether a user has rated 10 or more movies.  Users with too few ratings negatively impact the root mean square error of the ALS. The cutoff for the minimum number of movie ratings was set to 10 to coincide with the number of recommendations new users will be asked to make to get recommendations from the final recommendation engine. The new series is then checked for missing values and merged with the ratings dataframe. 

In [17]:
ratings_10 = ratings_df.groupby(['userId'])['movieId'].agg('count')>=10
ratings_10 = ratings_10.reset_index()
ratings_10.head()

,userId,movieId
0,1,True
1,2,True
2,3,True
3,4,True
4,5,True


In [18]:
ratings_10.isna().sum()

userId     0
movieId    0
dtype: int64

In [19]:
df_with_filter = ratings_df.merge(ratings_10, on='userId', how='left')
df_with_filter.head()

,userId,movieId_x,rating,movieId_y
0,1,307,3.5,True
1,1,481,3.5,True
2,1,1091,1.5,True
3,1,1257,4.5,True
4,1,1449,4.5,True


In [20]:
df_with_filter['movieId_y'].value_counts(), 

(True     27565774
 False      187670
 Name: movieId_y, dtype: int64,)

The next cell filters the ratings dataframe to create a dataframe that includes only those users with 10 or more recommendations. 

In [21]:
df_filtered = df_with_filter[df_with_filter['movieId_y']==True]
df_filtered.drop(['movieId_y'], axis=1, inplace=True)
df_filtered.columns = ['userId', 'movieId', 'rating']
df_filtered.head()

/anaconda3/envs/FI_final_project/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,userId,movieId,rating
0,1,307,3.5
1,1,481,3.5
2,1,1091,1.5
3,1,1257,4.5
4,1,1449,4.5


In the following cell, a final check is conducted to make sure that the number of users in the new dataframe plus the number of users with fewer than 10 ratings is equal to the total number of users in the original ratings dataframe.

In [24]:
df_filtered['userId'].nunique()+(ratings_df.groupby(['userId'])['movieId'].agg('count')<10).sum() == ratings_df['userId'].nunique()

True

Lastly, the new filtered dataframe is saved to the processed data folder.

In [ ]:
df_filtered.to_csv('../data/processed/ratings_processed.csv')